In [135]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.cluster import KMeans

# `Request Data from Census API`

In [136]:
# available datasets: https://api.census.gov/data.html

In [137]:
# get data from api
HOST = "https://api.census.gov/data"
year = "2018"
dataset = "acs/acs5"
base_url = "/".join([HOST, year, dataset])
predicates = {}
# request variables. definitions on census website
predicates["get"] = "NAME,B19083_001E,B19113_001E,B23025_003E,B23025_005E,B01001_001E,B25004_001E"
predicates["for"] = ('county', '*')
response = requests.get(base_url, params=predicates)
df = pd.read_json(response.text) 

# cleaning df
df. columns = df. iloc[0]
df = df. drop(0) 
df = df. reset_index(drop=True) 
df.columns = ['Location','gini_index','median_family_income','employed','unemployed','population','vacant_housing', 'state_id', 'county_id']
df

,Location,gini_index,median_family_income,employed,unemployed,population,vacant_housing,state_id,county_id
0,"Sedgwick County, Kansas",0.4554,69832,260607,13610,512064,21906,20,173
1,"Republic County, Kansas",0.4314,65655,2391,59,4686,634,20,157
2,"Graham County, Kansas",0.4176,57083,1191,42,2545,260,20,065
3,"Douglas County, Kansas",0.4714,82294,69358,3085,119319,3438,20,045
4,"Sheridan County, Kansas",0.4180,69637,1290,29,2506,123,20,179
...,...,...,...,...,...,...,...,...,...
3215,"Adams County, Idaho",0.4228,57050,1707,190,4019,1008,16,003
3216,"Jerome County, Idaho",0.4154,55933,11051,444,23431,622,16,053
3217,"Lewis County, Idaho",0.4586,53201,1609,94,3845,285,16,061
3218,"Owyhee County, Idaho",0.4761,49872,5198,234,11455,627,16,073


In [138]:
# create fips for mapping
df['fips'] = df.state_id + df.county_id

# convert str to number
df['gini_index'] = pd.to_numeric(df['gini_index'])
df['median_family_income'] = pd.to_numeric(df['median_family_income'])
df['employed'] = pd.to_numeric(df['employed'])
df['unemployed'] = pd.to_numeric(df['unemployed'])
df['population'] = pd.to_numeric(df['population'])
df['vacant_housing'] = pd.to_numeric(df['vacant_housing'])

# get percent unemployed
df['percent_unemployed'] = df.unemployed / df.employed * 100

In [139]:
# create new df of columns needed for analysis
dff = df[['gini_index','vacant_housing','percent_unemployed', 'median_family_income']]
dff.apply(pd.to_numeric)

,gini_index,vacant_housing,percent_unemployed,median_family_income
0,0.4554,21906,5.222423,69832.0
1,0.4314,634,2.467587,65655.0
2,0.4176,260,3.526448,57083.0
3,0.4714,3438,4.447937,82294.0
4,0.4180,123,2.248062,69637.0
...,...,...,...,...
3215,0.4228,1008,11.130639,57050.0
3216,0.4154,622,4.017736,55933.0
3217,0.4586,285,5.842138,53201.0
3218,0.4761,627,4.501731,49872.0


# `Distribution of Income`

In [140]:
fig = px.histogram (dff, x="median_family_income", nbins=100, marginal = 'box')
fig.show()

# `Distribution of Unemployment`

In [141]:
fig = px.histogram (dff, x="percent_unemployed", nbins=100, marginal = 'box')
fig.show()

# `Distribution of Income by State`

In [142]:
state = 'New York'
df_map = df[df['Location'].str.contains(state)]


fig = ff.create_choropleth(fips=df_map.fips,
    scope=[state],
    values=df_map.median_family_income,
    show_state_data=True,
    show_hover = True,
    title_text = f'{state} by Median Income',
    legend_title = 'Income Level')

fig.layout.template=None
    
fig.show()

# `Distribution of Unemployment by State`

In [143]:
state = 'New York'
df_map = df[df['Location'].str.contains(state)]


fig = ff.create_choropleth(fips=df_map.fips,
    scope=[state],
    values=df_map.percent_unemployed,
    show_state_data=True,
    show_hover = True,
    title_text = f'{state} by Unemployment',
    legend_title = '% Unemployed')

fig.layout.template=None
    
fig.show()

# `Distribution of Vacant Homes by State`

In [144]:
state = 'New York'
df_map = df[df['Location'].str.contains(state)]


fig = ff.create_choropleth(fips=df_map.fips,
    scope=[state],
    values=df_map.vacant_housing,
    show_state_data=True,
    show_hover = True,
    title_text = f'{state} by Vacancy Housing',
    legend_title = 'Vacant Houses')

fig.layout.template=None
    
fig.show()

In [145]:
# split up income into quartiles
# will use for last plot
dff['income_quartile'] = pd.qcut(dff['median_family_income'],q=4,labels=False,precision=0,duplicates='raise')
dff['income_quartile'] = dff['income_quartile'] + 1

dff.sort_values(by=['income_quartile'], inplace=True, ascending=True, na_position='first')
dff.dropna(subset = ['income_quartile'], inplace=True)
dff['income_quartile'] = dff['income_quartile'].astype(str)

# `Income vs Unemployment`

In [146]:
data = dff[['median_family_income','percent_unemployed']].to_numpy()

# k-clustering
# use k = 4

kmeans = KMeans(n_clusters=4)
kmeans.fit(data)
y_kmeans = kmeans.predict(data)

px.scatter(x=data[:, 0], y=data[:, 1], color=y_kmeans,
    labels={
            "x": "Median Income",
            "y": "Unemployment %",
            "color": "Cluster"
        })

# `Income vs Inequality`

In [147]:
data = dff[['median_family_income','gini_index']].to_numpy()

kmeans = KMeans(n_clusters=4)
kmeans.fit(data)
y_kmeans = kmeans.predict(data)

px.scatter(x=data[:, 0], y=data[:, 1], color=y_kmeans,
    labels={
            "x": "Median Income",
            "y": "Gini Index",
            "color": "Cluster"
        })

# `Scatter plot and regression by income quartile`
# `Enter x and y axes`

In [148]:
x_axis = 'gini_index'

y_axis = 'percent_unemployed'

fig = px.scatter(dff, x=x_axis, y=y_axis, facet_col='income_quartile', facet_col_wrap = 4, color='income_quartile', trendline='ols')
fig.show()